In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact

Populating the interactive namespace from numpy and matplotlib


# Example: Stability of Householder Triangularization
We want to perform an experiment to study the accuracy of the Householder QR algorithm. This will proceed in several steps.

## Step 1
We are going to construct a matrix $A$ where we known $Q$ and $R$ ahead of time (These will be close to the exact $Q$ and $R$ up to a little rounding error). 

## Step 2
We will then use the Householder QR algorithm to compute $Q_2$ and $R_2$. 

## Step 3
We will compute the error between $Q$ and our computed $Q_2$ from the Householder QR algorithm. We will do the same with $R$ and $R_2$. 

## Step 4
To test the **stability** of the algorithm, we will measure the error between the original $A$ and the computed $A_2 = Q_2R_2$.

In [2]:
## Step 1

n = 50

R = normal(0, 1, (n, n)) ## an nxn array of normal random variables (mean zero and variance one)
## We use loop to set the entries of R below the diagonal to zero
for i in arange(n):
    for j in arange(i):
        R[i, j] = 0
# We could replace the above with one line of code
# R = triu(normal(0, 1, (n, n))


## We need a way to construct a random orthogonal matrix, it needs to be orthogonal, and 
## we know that the SVD algorithm generates such a matrix
## (note that we only need the first output, so we discard the second 2 outputs)
Q, _, _ = svd(normal(0, 1, (n, n))) ## a random orthogonal matrix 

A = Q@R ## Our matrix A

In [3]:
## Step 2

Q2, R2 = qr(A)

## We need to match up the sign of the columns of Q with Q2 (and rows of R with R2)
s = sign(diag(R))*sign(diag(R2)) ## if the signs are different: -1, else: 1
Q2 = Q2*s[None, :] ## see the Python notebook on Broadcasting (Week 6)
R2 = R2*s[:, None]

In [4]:
## Step 3
print(norm(Q - Q2, 2)) ## why have we not normalized with norm(Q, 2)?
print(norm(R - R2, 2)/norm(R, 2))

0.14780504976994568
0.03168217093545392


In [5]:
## Step 4
A2 = Q2@R2
print(norm(A - A2, 2)/norm(A, 2))

6.706892958732318e-16


## What happened?
Even though we have relatively large differences $Q - Q_2$ and $R - R_2$, somehow the product $A_2 = Q_2R_2$ is very close to $A$. **There must be some miraculous cancelation of error when we mutlipy $Q_2R_2$.**

Let us try another experiment to emphasize this. We will generate two new matrices $Q_3$ and $R_3$ by adding a constant perturbation to $Q$ and $R$, that is, by a constant factor of $10^{-4}$ to simulate, roughly, the error we see between $Q$ and $Q_2$ and between $R$ and $R_2$. We will then look at the error between $A$ and the product $Q_3R_3$.

We expect to see the error between $A$ and $A_3$ to be around the size of the perturbation to $Q$ and $R$, i.e., $O(10^{-4})$.

In [6]:
Q3 = Q + 1e-4
R3 = R + 1e-4
print(norm(Q-Q3, 2))
print(norm(R-R3, 2)/norm(R, 2))

print('And now the error in the product...')
A3 = Q3@R3
print(norm(A - A3, 2)/norm(A, 2))

0.00499999999999984
0.0004005012288064835
And now the error in the product...
0.0016218672992371526


The above shows essentially what we expected to see. Why is this not true for $Q_2$ and $R_2$? Somehow the `qr()` function has given us inaccurate matrices $Q_2$ and $R_2$ that still contain, figuratively speaking, the essence of the matrix $A$.

# Solving nonsingular linear systems
What happens if we try to solve a linear system?

In [7]:
def backward_substitution(U, b):
    """Upper triangular matrix `U` and right hand side vector `b`. 
    Note that `U` must be square and `U` & `b` must have compatable shape."""
    b = array(b)
    assert b.size > 1
    assert b.ndim == 1 or b.ndim == 2
    n = b.size
    assert all(diag(U) != 0), 'the diagonal elements of U must be nonzero'
    assert U.ndim == 2
    assert all(array(U.shape) == n), 'the matrix U must be square and must have same number of rows as b'
    n = b.size
    x = ones_like(b)
    for k in arange(n)[::-1]: ## iterate in reverse order
        q = 0.
        for j in arange(k+1, n):
            q = q + U[k, j]*x[j]
        x[k] = (b[k] - q)/U[k, k]
    return x

In [11]:
b = ones((n, 1))

y = Q.T@b
y2 = Q2.T@b


print('Relative error between y and y2:')
print(norm(y - y2, 2)/norm(y))

print('Norm of vectors b, y, and y2')
print(norm(b, 2), norm(y, 2), norm(y2, 2))

print('Normalized residual using Q and y:')
print(norm(Q@y - b, 2)/norm(y, 2))
print('Normalized residual using Q2 and y2:')
print(norm(Q2@y2 - b, 2)/norm(y2, 2))

Relative error between y and y2:
0.02800527702785372
Norm of vectors b, y, and y2
7.0710678118654755 7.0710678118654755 7.071067811865475
Normalized residual using Q and y:
1.579094682910763e-15
Normalized residual using Q2 and y2:
6.246916271661717e-16


In [21]:
x = backward_substitution(R, y)
x2 = backward_substitution(R2, y2)

print('Relative error between x and x2:')
print(norm(x - x2, 2)/norm(x))

print('Norm of vectors x and x2')
print(norm(x, 2), norm(x2, 2))

print('Normalized residual using R and x:')
print(norm(R@x - y, 2)/norm(x, 2)/norm(R, 2))
print('Normalized residual using R2 and x2:')
print(norm(R2@x2 - y2, 2)/norm(x2, 2)/norm(R2, 2))

print('Condition number of R and R2')
print(cond(R), cond(R2))

Relative error between x and x2:
0.8725705538547797
Norm of vectors x and x2
1.5870246900567085e+17 2.971815702840701e+17
Normalized residual using R and x:
4.1557801778787144e-18
Normalized residual using R2 and x2:
4.762166365967782e-18
Condition number of R and R2
2.65559009375992e+18 2.5523488570018954e+18


In [26]:
print('Normalized residual using A and x:')
print(norm(A@x - b, 2)/norm(x, 2)/norm(A, 2))
print('Normalized residual using A2 and x2:')
print(norm(A2@x2 - b, 2)/norm(x2, 2)/norm(A, 2))

print('Condition number of A and A2')
print(cond(A), cond(A2))

Normalized residual using A and x:
6.596651548887683e-18
Normalized residual using A2 and x2:
9.788627322804369e-18
Condition number of A and A2
2.1807762629845178e+17 1.537995822493416e+17
